## Project 2: Expert

### A question answering agent that is an expert in a new product being launched

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy

In [ ]:
# imports

import os
from dotenv import load_dotenv
import gradio as gr

In [ ]:
# constants

MODEL = "gpt-3.5-turbo"

In [ ]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
# Load in the document

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

text_loader = TextLoader('product.txt')
loaded_data = text_loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_data = text_splitter.split_documents(loaded_data)

In [ ]:
# Put it into a vector store

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_data, embedding=embeddings)

In [ ]:
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=vectorstore.as_retriever(), memory=memory)

In [ ]:
# Let's try a simple question

query = "Please explain what ProsperAI is in a couple of sentences"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print(answer)

In [ ]:
# Wrapping that in a function

def expert(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(expert).launch()